In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
import joblib
import torch
import torch.nn as nn
import torch.optim as optim

In [19]:
df = pd.read_csv('station_day.csv')
df.head()

,StationId,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,AP001,2017-11-24,71.36,115.75,1.75,20.65,12.40,12.19,0.10,10.76,109.26,0.17,5.92,0.10,NaN,NaN
1,AP001,2017-11-25,81.40,124.50,1.44,20.50,12.08,10.72,0.12,15.24,127.09,0.20,6.50,0.06,184.0,Moderate
2,AP001,2017-11-26,78.32,129.06,1.26,26.00,14.85,10.28,0.14,26.96,117.44,0.22,7.95,0.08,197.0,Moderate
3,AP001,2017-11-27,88.76,135.32,6.60,30.85,21.77,12.91,0.11,33.59,111.81,0.29,7.63,0.12,198.0,Moderate
4,AP001,2017-11-28,64.18,104.09,2.56,28.07,17.01,11.42,0.09,19.00,138.18,0.17,5.02,0.07,188.0,Moderate


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108035 entries, 0 to 108034
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   StationId   108035 non-null  object 
 1   Date        108035 non-null  object 
 2   PM2.5       86410 non-null   float64
 3   PM10        65329 non-null   float64
 4   NO          90929 non-null   float64
 5   NO2         91488 non-null   float64
 6   NOx         92535 non-null   float64
 7   NH3         59930 non-null   float64
 8   CO          95037 non-null   float64
 9   SO2         82831 non-null   float64
 10  O3          82467 non-null   float64
 11  Benzene     76580 non-null   float64
 12  Toluene     69333 non-null   float64
 13  Xylene      22898 non-null   float64
 14  AQI         87025 non-null   float64
 15  AQI_Bucket  87025 non-null   object 
dtypes: float64(13), object(3)
memory usage: 13.2+ MB


In [21]:
learning_vars = ['PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3']
target_classification = 'AQI_Bucket'
target_regression = 'AQI'
fix_columns = ['PM2.5', 'PM10', 'NO2', 'CO', 'SO2', 'O3', 'AQI']

In [22]:
imputer = SimpleImputer(strategy='mean')
df[fix_columns] = imputer.fit_transform(df[fix_columns])

In [23]:
df.head()

,StationId,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,AP001,2017-11-24,71.36,115.75,1.75,20.65,12.40,12.19,0.10,10.76,109.26,0.17,5.92,0.10,179.74929,NaN
1,AP001,2017-11-25,81.40,124.50,1.44,20.50,12.08,10.72,0.12,15.24,127.09,0.20,6.50,0.06,184.00000,Moderate
2,AP001,2017-11-26,78.32,129.06,1.26,26.00,14.85,10.28,0.14,26.96,117.44,0.22,7.95,0.08,197.00000,Moderate
3,AP001,2017-11-27,88.76,135.32,6.60,30.85,21.77,12.91,0.11,33.59,111.81,0.29,7.63,0.12,198.00000,Moderate
4,AP001,2017-11-28,64.18,104.09,2.56,28.07,17.01,11.42,0.09,19.00,138.18,0.17,5.02,0.07,188.00000,Moderate


In [24]:
label_encoder = LabelEncoder()
df[target_classification] = label_encoder.fit_transform(df[target_classification])

In [25]:
df.head()

,StationId,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,AP001,2017-11-24,71.36,115.75,1.75,20.65,12.40,12.19,0.10,10.76,109.26,0.17,5.92,0.10,179.74929,6
1,AP001,2017-11-25,81.40,124.50,1.44,20.50,12.08,10.72,0.12,15.24,127.09,0.20,6.50,0.06,184.00000,1
2,AP001,2017-11-26,78.32,129.06,1.26,26.00,14.85,10.28,0.14,26.96,117.44,0.22,7.95,0.08,197.00000,1
3,AP001,2017-11-27,88.76,135.32,6.60,30.85,21.77,12.91,0.11,33.59,111.81,0.29,7.63,0.12,198.00000,1
4,AP001,2017-11-28,64.18,104.09,2.56,28.07,17.01,11.42,0.09,19.00,138.18,0.17,5.02,0.07,188.00000,1


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108035 entries, 0 to 108034
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   StationId   108035 non-null  object 
 1   Date        108035 non-null  object 
 2   PM2.5       108035 non-null  float64
 3   PM10        108035 non-null  float64
 4   NO          90929 non-null   float64
 5   NO2         108035 non-null  float64
 6   NOx         92535 non-null   float64
 7   NH3         59930 non-null   float64
 8   CO          108035 non-null  float64
 9   SO2         108035 non-null  float64
 10  O3          108035 non-null  float64
 11  Benzene     76580 non-null   float64
 12  Toluene     69333 non-null   float64
 13  Xylene      22898 non-null   float64
 14  AQI         108035 non-null  float64
 15  AQI_Bucket  108035 non-null  int64  
dtypes: float64(13), int64(1), object(2)
memory usage: 13.2+ MB


In [27]:
X = df[learning_vars]

y_class = df[target_classification]
y_reg = df[target_regression]

y_dual = df[[target_classification, target_regression]]

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)
X_train_dual, X_test_dual, y_train_dual, y_test_dual = train_test_split(X, y_dual, test_size=0.2, random_state=42)

In [31]:
scaler = StandardScaler()
X_train_class = scaler.fit_transform(X_train_class)
X_test_class = scaler.transform(X_test_class)
X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)
X_train_dual = scaler.fit_transform(X_train_dual)
X_test_dual = scaler.transform(X_test_dual)

In [32]:
X_train_class_tensor = torch.tensor(X_train_class, dtype=torch.float32)
y_train_class_tensor = torch.tensor(y_train_class.values, dtype=torch.long)
X_train_reg_tensor = torch.tensor(X_train_reg, dtype=torch.float32)
y_train_reg_tensor = torch.tensor(y_train_reg.values, dtype=torch.float32)
X_train_dual_tensor = torch.tensor(X_train_dual, dtype=torch.float32)
y_train_dual_tensor = torch.tensor(y_train_dual.values, dtype=torch.float32)

In [33]:
logistic_model = LogisticRegression(max_iter=200)
logistic_model.fit(X_train_class, y_train_class)
y_pred_class_logistic = logistic_model.predict(X_test_class)
print("Logistic Regression Classification Report:")
print(classification_report(y_test_class, y_pred_class_logistic))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.28      0.41      1098
           1       0.66      0.60      0.63      5867
           2       0.58      0.29      0.39      2338
           3       0.61      0.71      0.66      4697
           4       0.83      0.67      0.74      1051
           5       0.72      0.76      0.74      2391
           6       0.61      0.85      0.71      4165

    accuracy                           0.65     21607
   macro avg       0.68      0.60      0.61     21607
weighted avg       0.65      0.65      0.63     21607



In [35]:
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=0)
rf_classifier.fit(X_train_class, y_train_class)
y_pred_class_rf = rf_classifier.predict(X_test_class)
print("Random Forest Classification Report:")
print(classification_report(y_test_class, y_pred_class_rf))

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.69      0.73      1098
           1       0.78      0.81      0.79      5867
           2       0.66      0.63      0.65      2338
           3       0.76      0.83      0.79      4697
           4       0.83      0.82      0.82      1051
           5       0.77      0.79      0.78      2391
           6       0.97      0.86      0.91      4165

    accuracy                           0.80     21607
   macro avg       0.79      0.78      0.78     21607
weighted avg       0.80      0.80      0.80     21607



In [36]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg_rf = rf_regressor.predict(X_test_reg)
print("Random Forest Regression Metrics:")
print("MSE:", mean_squared_error(y_test_reg, y_pred_reg_rf))
print("R2 Score:", r2_score(y_test_reg, y_pred_reg_rf))

Random Forest Regression Metrics:
MSE: 1656.723178675042
R2 Score: 0.8824298696150571


In [42]:
class MLPClassifierTorch(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLPClassifierTorch, self).__init__()
        self.fc1 = nn.Linear(input_size, 200)
        self.fc2 = nn.Linear(200, 100)
        self.fc3 = nn.Linear(100, 50)
        self.fc4 = nn.Linear(50, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.softmax(self.fc4(x))
        return x

In [45]:
input_size = X_train_class.shape[1]
output_size = len(label_encoder.classes_)
mlp_classifier_torch = MLPClassifierTorch(input_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_classifier_torch.parameters(), lr=0.0001)

In [46]:
num_epochs = 100
for epoch in range(num_epochs):
    outputs = mlp_classifier_torch(X_train_class_tensor)
    loss = criterion(outputs, y_train_class_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 1.9424
Epoch [20/100], Loss: 1.9397
Epoch [30/100], Loss: 1.9368
Epoch [40/100], Loss: 1.9335
Epoch [50/100], Loss: 1.9295
Epoch [60/100], Loss: 1.9246
Epoch [70/100], Loss: 1.9185
Epoch [80/100], Loss: 1.9111
Epoch [90/100], Loss: 1.9021
Epoch [100/100], Loss: 1.8916


In [51]:
# mlp_classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, random_state=42)
# mlp_classifier.fit(X_train_class, y_train_class)
# y_pred_class_mlp = mlp_classifier_torch.predict(X_test_class)
y_pred_class_mlp = mlp_classifier_torch(X_train_class_tensor)
print("MLP Classification Report:")
print(classification_report(y_test_class, y_pred_class_mlp))

MLP Classification Report:


ValueError: Found input variables with inconsistent numbers of samples: [21607, 86428]

In [50]:
mlp_regressor = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=300, random_state=42)
mlp_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg_mlp = mlp_regressor.predict(X_test_reg)
print("MLP Regression Metrics:")
print("MSE:", mean_squared_error(y_test_reg, y_pred_reg_mlp))
print("R2 Score:", r2_score(y_test_reg, y_pred_reg_mlp))

MLP Regression Metrics:
MSE: 1699.397994992062
R2 Score: 0.879401431440758


In [51]:
y_dual = df[[target_classification, target_regression]]
X_train_dual, X_test_dual, y_train_dual, y_test_dual = train_test_split(X, y_dual, test_size=0.2, random_state=42)

X_train_dual = scaler.fit_transform(X_train_dual)
X_test_dual = scaler.transform(X_test_dual)

mlp_dual = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=300, random_state=42)
mlp_dual.fit(X_train_dual, y_train_dual)
y_pred_dual = mlp_dual.predict(X_test_dual)

print("MLP Dual Prediction Metrics:")
print("MSE (AQI_Bucket):", mean_squared_error(y_test_dual[target_classification], y_pred_dual[:, 0]))
print("MSE (AQI):", mean_squared_error(y_test_dual[target_regression], y_pred_dual[:, 1]))

MLP Dual Prediction Metrics:
MSE (AQI_Bucket): 1.987573014570189
MSE (AQI): 1690.2473674677003


In [59]:
joblib.dump(logistic_model, 'logistic_model.pkl')
joblib.dump(rf_classifier, 'rf_classifier.pkl')
joblib.dump(rf_regressor, 'rf_regressor.pkl')
joblib.dump(mlp_classifier, 'mlp_classifier.pkl')
joblib.dump(mlp_regressor, 'mlp_regressor.pkl')
joblib.dump(mlp_dual, 'mlp_dual.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [60]:
def predict_models(x_new_test):
    # Load models and scaler
    logistic_model = joblib.load('logistic_model.pkl')
    rf_classifier = joblib.load('rf_classifier.pkl')
    rf_regressor = joblib.load('rf_regressor.pkl')
    mlp_classifier = joblib.load('mlp_classifier.pkl')
    mlp_regressor = joblib.load('mlp_regressor.pkl')
    mlp_dual = joblib.load('mlp_dual.pkl')
    scaler = joblib.load('scaler.pkl')
    label_encoder = joblib.load('label_encoder.pkl')

    # Scale the custom data
    x_new_test_scaled = scaler.transform(x_new_test)

    # Make predictions
    logistic_pred = logistic_model.predict(x_new_test_scaled)
    logistic_label = label_encoder.inverse_transform(logistic_pred)

    rf_class_pred = rf_classifier.predict(x_new_test_scaled)
    rf_class_label = label_encoder.inverse_transform(rf_class_pred)

    rf_reg_pred = rf_regressor.predict(x_new_test_scaled)

    mlp_class_pred = mlp_classifier.predict(x_new_test_scaled)
    mlp_class_label = label_encoder.inverse_transform(mlp_class_pred)

    mlp_reg_pred = mlp_regressor.predict(x_new_test_scaled)

    mlp_dual_pred = mlp_dual.predict(x_new_test_scaled)
    mlp_dual_class_label = label_encoder.inverse_transform(mlp_dual_pred[:, 0].astype(int))
    mlp_dual_reg_pred = mlp_dual_pred[:, 1]

    # Print predictions
    print("Logistic Regression Prediction:", logistic_label)
    print("Random Forest Classification Prediction:", rf_class_label)
    print("Random Forest Regression Prediction:", rf_reg_pred)
    print("MLP Classification Prediction:", mlp_class_label)
    print("MLP Regression Prediction:", mlp_reg_pred)
    print("MLP Dual Prediction - Classification Label:", mlp_dual_class_label)
    print("MLP Dual Prediction - Regression Value:", mlp_dual_reg_pred)


In [65]:
custom_test_data = {
    'PM2.5': [21],
    'PM10': [19],
    'NO2': [8],
    'CO': [1],
    'SO2': [1],
    'O3': [10]
}
x_new_test = pd.DataFrame(custom_test_data)

In [66]:
predict_models(x_new_test)

Logistic Regression Prediction: ['Satisfactory']
Random Forest Classification Prediction: ['Satisfactory']
Random Forest Regression Prediction: [80.76991485]
MLP Classification Prediction: ['Satisfactory']
MLP Regression Prediction: [66.30558873]
MLP Dual Prediction - Classification Label: ['Moderate']
MLP Dual Prediction - Regression Value: [64.0536822]
